In [1]:
import tensorflow as tf
import numpy as np
print("TensorFlow version: ",format(tf.__version__))
from pathlib import Path

TensorFlow version:  2.0.0


### chargement des données

In [2]:
# nombre d images
nbdata = 1000
# chemin vers les données
trainDataFile = Path("/home/romain/jupyter_mla/Databases/data_1k.bin")
LabelFile = Path("/home/romain/jupyter_mla/Databases/gender_1k.bin")

In [3]:
# dimension des images (de l'input x): 2304 car taille des images 48*48 pixels en niveau de gris
dim = 2304
# ouverture du fichier données
f = open(trainDataFile, 'rb')
# ouverture d'un fichier vide
data = np.empty([nbdata, dim], dtype=np.float32)
# chargement des données ligne par ligne
for i in range(nbdata):
    data[i, :] = np.fromfile(f, dtype=np.uint8, count=dim).astype(np.float32)
# fermeture du fichier
f.close()

In [4]:
# idem avec les étiquettes
f = open(LabelFile, 'rb')
label = np.empty([nbdata, 2], dtype=np.float32)
for i in range(nbdata):
    label[i, :] = np.fromfile(f, dtype=np.float32, count=2)
f.close()

### classes pour définir les couches et la structure du réseau

note: la classe 'Module' de tensorflow sert à créer un conteneur générique. Ce conteneur peut engranger des variables, des fonctions, et même d'autres modules; de manière générale, la calsse Module peut contenir tout objet sur lequel on peutt envoyer des inputs pour les travailler.

note: la méthode "call" permet d'utiliser une classe comme une méthode. Il faut pour celà d'abord déclarer
une instance de la classe concernée: par exemple fc1=fc_layer(...).
On peut ensuite faire un appel de méthode en utilisant l'instance ainsi créée comme si elle était une méthode: par exemple fc1(x).
Cela aura pour résultat d'appeller la méthode appelé __call__ au sein de la classe fc_layer et de l'exécuter avec l'input envoyé en entrée.

In [5]:
class fc_layer(tf.Module):
    def __init__(self, input_dim, output_dim):
        w_init = tf.random.truncated_normal([input_dim, output_dim], stddev=0.1)
        self.w = tf.Variable(w_init)
        print('w      ', self.w.get_shape())
        b_init = tf.constant(0.0, shape=[output_dim])
        self.b = tf.Variable(b_init)
        print('b      ', self.b.get_shape())

    def __call__(self, x):
        return tf.matmul(x, self.w) + self.b

In [6]:
class SimpleNet(tf.Module):
    def __init__(self, input_dim):
        self.fc1 = fc_layer(input_dim,50)
        self.fc2 = fc_layer(50,2)

    def __call__(self, x):
        x = self.fc1(x)
        x = tf.nn.sigmoid(x)
        x = self.fc2(x)
        return x

### méthodes pour entrainer le modèle

In [7]:
def train_one_step(model, optimizer, image, label):
    with tf.GradientTape() as tape:
        y = model(image)
        loss = tf.reduce_sum(tf.square(y - label))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [8]:
optimizer = tf.optimizers.SGD(1e-5)
simple_model = SimpleNet(dim)

w       (2304, 50)
b       (50,)
w       (50, 2)
b       (2,)


In [9]:
curPos = 0
batchSize = 256

In [10]:
for it in range(5000):
    if curPos + batchSize > nbdata:
        curPos = 0
    loss = train_one_step(simple_model, optimizer,
                          data[curPos:curPos + batchSize, :], label[curPos:curPos + batchSize, :])
    curPos += batchSize
    if it % 100 == 0 or it < 10:
        print("it= %6d - loss= %f" % (it, loss.numpy()))

it=      0 - loss= 233.125168
it=      1 - loss= 291.884491
it=      2 - loss= 172.478394
it=      3 - loss= 223.546082
it=      4 - loss= 181.255508
it=      5 - loss= 119.982124
it=      6 - loss= 224.515259
it=      7 - loss= 132.087723
it=      8 - loss= 97.573235
it=      9 - loss= 232.923309
it=    100 - loss= 77.959435
it=    200 - loss= 58.542015
it=    300 - loss= 217.838104
it=    400 - loss= 74.132187
it=    500 - loss= 49.015774
it=    600 - loss= 179.988770
it=    700 - loss= 58.728489
it=    800 - loss= 49.096252
it=    900 - loss= 164.612396
it=   1000 - loss= 59.599312
it=   1100 - loss= 52.093494
it=   1200 - loss= 145.187836
it=   1300 - loss= 52.506577
it=   1400 - loss= 52.320679
it=   1500 - loss= 118.220573
it=   1600 - loss= 48.077209
it=   1700 - loss= 42.157368
it=   1800 - loss= 90.892944
it=   1900 - loss= 53.641838
it=   2000 - loss= 41.738533
it=   2100 - loss= 101.725723
it=   2200 - loss= 51.061981
it=   2300 - loss= 43.185810
it=   2400 - loss= 86.229271